In [1]:
import numpy as np
import matplotlib.pylab as plt
import pickle

from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

本章我们将学习一个能够高效计算权重参数的梯度的方法——误差反向传播法。

要正确理解误差反向传播法，我个人认为有两种方法：
- 一种是基于数学式；
- 另一种是基于计算图（computational graph）。

前者是比较常见的方法，机器学习相关的图书中多数都是以数学式为中心展开论述的。因为这种方法严密且简洁，所以确实非常合理，但如果一上来就围绕数学式进行探讨，会忽略一些根本的东西，止步于式子的罗列。因此，本章希望大家通过计算图，直观地理解误差反向传播法。然后，再结合实际的代码加深理解，相信大家一定会有种“原来如此！”的感觉。

# 计算图

<img src="img/5_3.png" alt="Drawing" style="width: 500px;"/>


综上，用计算图解题的情况下，需要按如下流程进行。

1. 构建计算图。
2. 在计算图上，从左向右进行计算。

- 这里的第2歩“从左向右进行计算”是一种正方向上的传播，简称为正向传播（forward propagation）。正向传播是从计算图出发点到结束点的传播。
- 既然有正向传播这个名称，当然也可以考虑反向（从图上看的话，就是从右向左）的传播。实际上，这种传播称为反向传播（backward propagation）。反向传播将在接下来的导数计算中发挥重要作用。

计算图的特征是可以通过传递**“局部计算”**获得最终结果。“局部”这个词的意思是“与自己相关的某个小范围”。局部计算是指，无论全局发生了什么，都能只根据与自己相关的信息输出接下来的结果。

虽然局部计算非常简单，但是通过传递它的计算结果，可以获得全局的复杂计算的结果。

**为何用计算图解题？**

- 一个优点就在于前面所说的局部计算。无论全局是多么复杂的计算，都可以通过局部计算使各个节点致力于简单的计算，从而简化问题。
- 另一个优点是，利用计算图可以将中间的计算结果全部保存起来（比如，计算进行到2个苹果时的金额是200日元、加上消费税之前的金额650日元等）。
- **实际上，使用计算图最大的原因是，可以通过反向传播高效计算导数。**

<img src="img/5_5.png" alt="Drawing" style="width: 500px;"/>

如图5-5所示，反向传播使用与正方向相反的箭头（粗线）表示。反向传播传递“局部导数”，将导数的值写在箭头的下方。

在这个例子中，反向传播从右向左传递导数的值（1 → 1.1 → 2.2）。从这个结果中可知，“支付金额关于苹果的价格的导数”的值是2.2。

这意味着，如果苹果的价格上涨1日元，最终的支付金额会增加2.2日元（严格地讲，如果苹果的价格增加某个微小值，
则最终的支付金额将增加那个微小值的2.2倍）。

这里只求了关于苹果的价格的导数，不过“支付金额关于消费税的导数”“支付金额关于苹果的个数的导数”等也都可以用同样的方式算出来。并且，计算中途求得的导数的结果（中间传递的导数）可以被共享，从而可以高效地计算多个导数。

综上，计算图的优点是，可以通过正向传播和反向传播高效地计算各个变量的导数值。


# 链式法则

链式法则是关于复合函数的导数的性质，定义如下：

> 如果某个函数由复合函数表示，则该复合函数的导数可以用构成复合函数的各个函数的导数的乘积表示。

从下面图例可以简单理解：链式法则就是利用复合函数的乘积定义，来一层一层向上游求导。

<img src="img/5_5_2.png" alt="Drawing" style="width: 500px;"/>


<img src="img/5_7.png" alt="Drawing" style="width: 500px;"/>


# 反向传播

## 加法节点的反向传播

<img src="img/5_9.png" alt="Drawing" style="width: 500px;"/>

因为加法节点的反向传播只乘以1，所以输入的值会原封不动地流向下一个节点。

**简单理解就是：遇到加法直接把“右侧的导出”传递到“左侧”即可。**

## 乘法节点的反向传播

<img src="img/5_12.png" alt="Drawing" style="width: 500px;"/>

**乘法的反向传播会将上游的值乘以正向传播时的输入信号的“翻转值”后传递给下游。**

翻转值表示一种翻转关系，如图5-12所示，正向传播时信号是x的话，反向传播时则是y；正向传播时信号是y的话，反向传播时则是x。


# 简单层的实现

这里对以上两个节点：“加法节点”和“乘法节点”进行简单的代码实现。

## 乘法层

In [2]:
# 乘法层的实现
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
        
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        return out
    
    def backward(self, dout):
        dx = dout * self.y # 翻转x和y
        dy = dout * self.x
        return dx, dy


In [3]:
# 乘法层实例测试
apple = 100
apple_num = 2
tax = 1.1

# layer
mul_apple_layer = MulLayer() # 评估总价计算层
mul_tax_layer = MulLayer() # 总价税收计算层
# 每一层都会生成一个类对象，其中之一的原因是为了保存这一层的x和y值，后面会用来进行”反向传播“（求导）

# forward
# cc：在这一步除了正向传播计算之外，通过类变量的方式存储了每一层的x 和 y，这样在执行下面backward的时候，就可以计算出对应“导数”结果；
apple_price = mul_apple_layer.forward(apple, apple_num)
print(apple_price)
price = mul_tax_layer.forward(apple_price, tax)
print(price) # 220

# 上面两个print结果好理解：
# 1. 2个苹果总价格
# 2. 苹果总价 + 税收 之后的总价格


# backward
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
print(dapple_price, dtax)

dapple, dapple_num = mul_apple_layer.backward(dapple_price)
print(dapple, dapple_num, dtax) # 2.2 110 200

# 结果如下图

200
220.00000000000003
1.1 200
2.2 110.00000000000001 200


结果如图：


<img src="img/5_13.png" alt="Drawing" style="width: 500px;"/>


## 加法层

In [4]:
class AddLayer:
    def __init__(self):
        pass
    
    def forward(self, x, y):
        out = x + y
        return out
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy
    

In [5]:
# 加法层 + 乘法层 实例测试
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# layer
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num) #(1)
orange_price = mul_orange_layer.forward(orange, orange_num) #(2)
all_price = add_apple_orange_layer.forward(apple_price, orange_price) #(3)
price = mul_tax_layer.forward(all_price, tax) #(4)

# backward
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice) #(4)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price) #(3)
dorange, dorange_num = mul_orange_layer.backward(dorange_price) #(2)
dapple, dapple_num = mul_apple_layer.backward(dapple_price) #(1)

print(price) # 715
print(dapple_num, dapple, dorange, dorange_num, dtax) # 110 2.2 3.3 165 650



715.0000000000001
110.00000000000001 2.2 3.3000000000000003 165.0 650


# 激活函数层的实现

现在，我们将计算图的思路应用到神经网络中。这里，我们把构成神经网络的层实现为一个类。先来实现激活函数的ReLU层和Sigmoid层。

## ReLU层

<img src="img/f_5_7.png" alt="Drawing" style="width: 500px;"/>


<img src="img/5_18.png" alt="Drawing" style="width: 500px;"/>


In [6]:
class Relu:
    def __init__(self):
        self.mask = None
         
    # x 传进来的是一个numpy数组；
    # mask 就成了x中小于等于0为True的标记位；
    # 然后让out = x之后，对out为mask标记位为True置为0，那么其他为False 为原值
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        return out
    
    # dout同理，mask标记位为True的，dout改为0，其他不变
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        return dx
    
    

In [7]:
# mask标记位的代码解释
x = np.array([[1.0, -0.5], [-2.0, 3.0]])
print(x)
mask = (x <= 0)
print(mask)

# 上文代码中，会将为True的地方（<=0）设置为0 

[[ 1.  -0.5]
 [-2.   3. ]]
[[False  True]
 [ True False]]


In [8]:
# Relu 测试
relu_test = Relu()
a = relu_test.forward(x)
a
dout = np.array([[1, 8], [1, 1]])
b = relu_test.backward(dout)
b

array([[1., 0.],
       [0., 3.]])

array([[1, 0],
       [0, 1]])

## Sigmoid 层

<img src="img/5_19.png" alt="Drawing" style="width: 500px;"/>

<img src="img/5_20.png" alt="Drawing" style="width: 500px;"/>

这个值只根据正向传播时的输入x和输出y就可以算出来。因此，图5-20的计算图可以画成图5-21的集约化的“sigmoid”节点。
<img src="img/5_21.png" alt="Drawing" style="width: 500px;"/>

经过公式推导之后，得到这样的结果，这样就更方便代码实现了。
<img src="img/5_22.png" alt="Drawing" style="width: 500px;"/>


In [9]:
class Sigmoid:
    def __init__(self):
        self.out = None
        
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        return dx
    
    

In [10]:
# test
sig_test = Sigmoid()
x = np.array([[1.0, -0.5], [-2.0, 3.0]])
forward = sig_test.forward(x)
print(forward)

dout = np.array([[1, 1], [1, 1]])
backward = sig_test.backward(dout)
print(backward)


[[0.73105858 0.37754067]
 [0.11920292 0.95257413]]
[[0.19661193 0.23500371]
 [0.10499359 0.04517666]]


# Affine/Softmax层的实现

## Affine 层

神经网络的正向传播中，为了计算加权信号的总和，使用了矩阵的乘积运算。

这个乘积运算在几何学领域被称为“仿射变换”。因此，这里将进行仿射变换的处理实现为“Affine层”。关于仿射变换的几何数学解释可以在链接查看：https://www.matongxue.com/madocs/244/

- 将乘积运算用“dot”节点表示的话，则np.dot(X, W) + B的运算可用图5-24所示的计算图表示出来。
- 另外，在各个变量的上方标记了它们的形状（比如，计算图上显示了X的形状为(2,)，X·W的形状为(3,)等）。
- 现在这里的各个节点传播的是矩阵，之前我们看到的都是标量


<img src="img/5_24.png" alt="Drawing" style="width: 500px;"/>


<img src="img/5_25.png" alt="Drawing" style="width: 500px;"/>

<img src="img/5_27.png" alt="Drawing" style="width: 500px;"/>


In [11]:

# 和偏置B有关一点知识
X_dot_W = np.array([[0, 0, 0], [10, 10, 10]])
B = np.array([1, 2, 3])   
X_dot_W
X_dot_W + B

# 正向传播时，偏置被加到X·W的各个数据上。cc：这个很好理解。

# 反向传播时，各个数据的反向传播的值需要汇总为偏置的元素。cc： 这个就不好理解了。
dY = np.array([[1, 2, 3,], [4, 5, 6]])
dY

dB = np.sum(dY, axis=0)
dB
 
# 可以参考在5-27的第3个公式，但是还是没有理解。先过~

array([[ 0,  0,  0],
       [10, 10, 10]])

array([[ 1,  2,  3],
       [11, 12, 13]])

array([[1, 2, 3],
       [4, 5, 6]])

array([5, 7, 9])

In [12]:
# 批版本的Affine层
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
        
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        return dx
    

## Softmax-with-Loss 层

在第3章讲到，softmax函数会将输入值正规化之后再输出。

<img src="img/5_28.png" alt="Drawing" style="width: 500px;"/>

这里把第3章的神经网络拿出来做个类比：（上下两图刚好都是3层：Affine+ReLU一组为一层）

- x，w，b的运算为Affine层，就是通过dot做了”仿射变换“
- h()对应这里的ReLU层（如果激活函数用的ReLU，那也可以是Sigmoid函数）
<img src="img/3_18_copy.png" alt="Drawing" style="width: 300px;"/>


神经网络中进行的处理有推理（inference）和学习两个阶段。
- 神经网络的推理通常不使用 Softmax层。
    比如，用图 5-28的网络进行推理时，会将最后一个 Affine层的输出作为识别结果。神经网络中未被正规化的输出结果（图 5-28中 Softmax层前面的 Affine层的输出）有时被称为“得分”。也就是说，当神经网络的推理只需要给出一个答案的情况下，因为此时只对得分最大值感兴趣，所以不需要 Softmax层。
- 不过，神经网络的学习阶段则需要 Softmax层。

考虑到这里也包含作为损失函数的交叉熵误差（cross entropy error），所以称为“Softmax-with-Loss层”

<img src="img/5_29.png" alt="Drawing" style="width: 500px;"/>

<img src="img/5_30.png" alt="Drawing" style="width: 500px;"/>

- softmax函数记为Softmax层，交叉熵误差记为Cross Entropy Error层。
- 这里假设要进行3类分类，从前面的层接收3个输入（得分）。如图5-30所示，Softmax层将输入（a1, a2, a3）正规化，输出（y1, y2, y3）。
- Cross Entropy Error层接收Softmax的输出（y1, y2, y3）和标签（t1, t2, t3），从这些数据中输出损失L。



**重点：**

图5-30中要注意的是反向传播的结果。Softmax层的反向传播得到了（y1 − t1, y2 − t2, y3 − t3）这样“漂亮”的结果（公式这里就不推导了）。由于（y1, y2, y3）是Softmax层的输出，（t1, t2, t3）是监督数据，所以（y1 − t1, y2 − t2, y3 − t3）是Softmax层的输出和标签的差分。**神经网络的反向传播会把这个差分表示的误差传递给前面的层，这是神经网络学习中的重要性质。**

神经网络学习的目的就是通过调整权重参数，使神经网络的输出（Softmax的输出）接近标签。因此，必须将神经网络的输出与教师标签的误差高效地传递给前面的层。刚刚的（y1 − t1, y2 − t2, y3 − t3）正是Softmax层的输出与标签的差，直截了当地表示了当前神经网络的输出与标签的误差。

使用交叉熵误差作为 softmax函数的损失函数后，反向传播得到（y1 − t1, y2 − t2, y3 − t3）这样“漂亮”的结果。实际上，这样“漂亮”的结果并不是偶然的，而是为了得到这样的结果，特意设计了交叉熵误差函数。回归问题中输出层使用“恒等函数”，损失函数使用“平方和误差”，也是出于同样的理由（3.5节）。也就是说，使用“平方和误差”作为“恒等函数”的损失函数，反向传播才能得到（y1 − t1, y2 − t2, y3 − t3）这样“漂亮”的结果。

In [13]:
# 代码实现
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None # 损失
        self.y = None # softmax的输出
        self.t = None # 监督数据（one-hot vector）
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        return dx


这里考虑一个具体的例子，比如标签是（0, 1, 0），Softmax层的输出是(0.3, 0.2, 0.5)的情形。因为正确解标签处的概率是0.2（20%），这个时候的神经网络未能进行正确的识别。此时，Softmax层的反向传播传递的是(0.3, −0.8, 0.5)这样一个大的误差。因为这个大的误差会向前面的层传播，所以Softmax层前面的层会从这个大的误差中学习到“大”的内容。

再举一个例子，比如标签是(0, 1, 0)，Softmax层的输出是(0.01, 0.99, 0)的情形（这个神经网络识别得相当准确）。此时Softmax层的反向传播传递的是(0.01, −0.01, 0)这样一个小的误差。这个小的误差也会向前面的层传播，因为误差很小，所以Softmax层前面的层学到的内容也很“小”。

# 误差反向传播法的实现

在进行具体的实现之前，我们再来确认一下神经网络学习的全貌图。神经网络学习的步骤如下所示。

**前提**

神经网络中有合适的权重和偏置，调整权重和偏置以便拟合训练数据的过程称为学习。神经网络的学习分为下面4个步骤。

- 步骤1（mini-batch）
    从训练数据中随机选择一部分数据。
- 步骤2（计算梯度）
    计算损失函数关于各个权重参数的梯度。
- 步骤3（更新参数）
    将权重参数沿梯度方向进行微小的更新。
- 步骤4（重复）
    重复步骤1、步骤2、步骤3。
    
误差反向传播法会在步骤2中出现。上一章中，我们利用数值微分求得了这个梯度。数值微分虽然实现简单，但是计算要耗费较多的时间。和需要花费较多时间的数值微分不同，误差反向传播法可以快速高效地计算梯度。

## 误差反向传播法的神经网络的实现

In [3]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict


class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # 初始化参数，是一个字典。
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
        # 生成层
        self.layers = OrderedDict() # cc：有序字典：“有序”是指它可以记住向字典里添加元素的顺序。
        # cc：以layers['Affine1']、layers['ReLu1']、layers['Affine2']的形式，通过有序字典保存各个层
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1']) # 把Affine 对象存到layers里面
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x) # cc：求出每一层的output，但是只返回最后一次的结果。中间层的结果不返回，但是会保存其类变量，供反向传播使用；
        return x

    # x:输入数据, t:监督数据
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
            
    # x:输入数据, t:监督数据
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        return grads
    
    def gradient(self, x, t):
        # forward
        self.loss(x, t)
        
        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)
        layers = list(self.layers.values())
        layers.reverse() # 这里对layers做了反序，则 开始反向传播，调取每个对象的backward方法
        for layer in layers:
            dout = layer.backward(dout)
            
        # 设定
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db
        return grads
    

像这样通过将神经网络的组成元素以层的方式实现，可以轻松地构建神经网络。这个用层进行模块化的实现具有很大优点。因为想另外构建一个神经网络（比如5层、10层、20层……的大的神经网络）时，只需像组装乐高积木那样添加必要的层就可以了。之后，通过各个层内部实现的正向传播和反向传播，就可以正确计算进行识别处理或学习所需的梯度。

## 误差反向传播法的梯度验证正确性

In [5]:
# 误差反向传播法的梯度确认: 微分法 和 解析法 求差值；
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

# 读入数据
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label = True)
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)
x_batch = x_train[:3]

t_batch = t_train[:3]
grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

# 求各个权重的绝对误差的平均值
for key in grad_numerical.keys():
    diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key]))
    print(key + ":" + str(diff))
#     print('grad_numerical:',grad_numerical[key])
#     print('grad_backprop:', grad_backprop[key])

# 从这个结果可以看出，通过数值微分和误差反向传播法求出的梯度的差非常小。

W1:5.080372264776923e-10
b1:2.8296791743684206e-09
W2:5.2661368126302625e-09
b2:1.4004956144508806e-07


数值微分和误差反向传播法的计算结果之间的误差为 0是很少见的。这是因为计算机的计算精度有限（比如，32位浮点数）。受到数值精度的限制，刚才的误差一般不会为 0，但是如果实现正确的话，可以期待这个误差是一个接近 0的很小的值。如果这个值很大，就说明误差反向传播法的实现存在错误。



## 案例手写数字：使用误差反向传播法的学习


In [7]:
# 使用误差反向传播法的学习
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet
# 读入数据
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)
iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1
train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 通过误差反向传播法求梯度
    grad = network.gradient(x_batch, t_batch)
    
    # 更新：每轮都会更新这4个参数值
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
        
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)


0.11775 0.1181
0.9046666666666666 0.9081
0.92405 0.927
0.93465 0.9343
0.9459 0.9445
0.9513166666666667 0.9484
0.9578666666666666 0.9532
0.9617333333333333 0.9567
0.9657 0.9602
0.96895 0.9625
0.9718 0.9665
0.9732 0.9677
0.9740833333333333 0.9668
0.975 0.9684
0.97795 0.9705
0.9791333333333333 0.9702
0.9801333333333333 0.9702


# 总结

本章所学的内容
- 通过使用计算图，可以直观地把握计算过程。
- 计算图的节点是由局部计算构成的。局部计算构成全局计算。
- 计算图的正向传播进行一般的计算。通过计算图的反向传播，可以计算各个节点的导数。
- 通过将神经网络的组成元素实现为层，可以高效地计算梯度（反向传播法）。
- 通过比较数值微分和误差反向传播法的结果，可以确认误差反向传播法的实现是否正确（梯度确认）。